# Supervised Learning Capstone 

Dataset: https://www.kaggle.com/austinreese/craigslist-carstrucks-data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from wordcloud import WordCloud

# Display preferences
%matplotlib inline
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 2000)

In [2]:
# Create dataframe
cars = pd.read_csv('../../data/vehicles.csv')

## Data Cleaning 

To begin, we'll start by taking a look at the dimensions of this dataset.

In [3]:
print('Number of rows: {}'.format(cars.shape[0]))
print('Number of columns: {}'.format(cars.shape[1]))

Number of rows: 509577
Number of columns: 25


Let's now review the column types and missing values.

In [4]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509577 entries, 0 to 509576
Data columns (total 25 columns):
id              509577 non-null int64
url             509577 non-null object
region          509577 non-null object
region_url      509577 non-null object
price           509577 non-null int64
year            508050 non-null float64
manufacturer    486813 non-null object
model           501588 non-null object
condition       277643 non-null object
cylinders       309894 non-null object
fuel            505592 non-null object
odometer        417253 non-null float64
title_status    506515 non-null object
transmission    505858 non-null object
vin             302152 non-null object
drive           365434 non-null object
size            167574 non-null object
type            368046 non-null object
paint_color     344871 non-null object
image_url       509563 non-null object
description     509561 non-null object
county          0 non-null float64
state           509577 non-null obj

There are a total of 25 columns in this dataset: 5 are of type float, 2 are of type integer and 18 are of type string.

In [5]:
cars.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
0,7034441763,https://saltlakecity.craigslist.org/cto/d/salt...,salt lake city,https://saltlakecity.craigslist.org,17899,2012.0,volkswagen,golf r,excellent,4 cylinders,gas,63500.0,clean,manual,WVWPF7AJ6CW316713,4wd,compact,hatchback,black,https://images.craigslist.org/00G0G_fTLDWM5Xyv...,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,gas,10.0,clean,automatic,1FTMF1EP3GKF13544,4wd,NaN,NaN,NaN,https://images.craigslist.org/00v0v_7Cu0buIofU...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,46463,2015.0,gmc,sierra 1500,excellent,NaN,gas,7554.0,clean,automatic,3GTU2WEC6FG228025,4wd,NaN,NaN,white,https://images.craigslist.org/01515_lPvJ9bfbdY...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,https://saltlakecity.craigslist.org/ctd/d/sand...,salt lake city,https://saltlakecity.craigslist.org,0,2016.0,ford,f-150,excellent,NaN,gas,10.0,clean,automatic,1FTEX1EF6GKD25447,4wd,NaN,NaN,NaN,https://images.craigslist.org/00T0T_6Rjfp3NS4O...,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,49999,2018.0,ford,f-450,NaN,NaN,diesel,70150.0,clean,automatic,1FT8W4DT8GEA90427,4wd,NaN,pickup,white,https://images.craigslist.org/00W0W_8yIUwRBXXd...,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [6]:
cars.tail()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,county,state,lat,long
509572,7034554007,https://saltlakecity.craigslist.org/ctd/d/engl...,salt lake city,https://saltlakecity.craigslist.org,15476,2012.0,nissan,xterra,NaN,6 cylinders,gas,103135.0,clean,automatic,5N1AN0NW0CC520698,4wd,NaN,SUV,NaN,https://images.craigslist.org/00z0z_3ERBmVoD5s...,Maximum Auto SearchÂ Â Â Â Â Â Â Â STOCK #:Â Â...,NaN,ut,39.6405,-104.961
509573,7034538385,https://saltlakecity.craigslist.org/ctd/d/engl...,salt lake city,https://saltlakecity.craigslist.org,9881,2008.0,bmw,3 series 328i,NaN,6 cylinders,gas,108277.0,clean,automatic,WBAWR33558P150498,rwd,NaN,convertible,NaN,https://images.craigslist.org/01313_38rMiNAW18...,Maximum Auto SearchÂ Â Â Â Â Â Â Â STOCK #:Â Â...,NaN,ut,39.6405,-104.961
509574,7034523643,https://saltlakecity.craigslist.org/ctd/d/salm...,salt lake city,https://saltlakecity.craigslist.org,24895,2019.0,jeep,cherokee,NaN,4 cylinders,gas,6973.0,clean,automatic,1C4PJMLB0KD267506,4wd,NaN,SUV,white,https://images.craigslist.org/00606_dOtGS5RmTi...,"2019 Jeep Cherokee Latitude Plus -- $24,895 ...",NaN,ut,45.1637,-113.894
509575,7034458858,https://saltlakecity.craigslist.org/cto/d/ogde...,salt lake city,https://saltlakecity.craigslist.org,32500,2017.0,NaN,Porsche Macan GTS,excellent,6 cylinders,gas,37520.0,clean,automatic,NaN,NaN,NaN,NaN,red,https://images.craigslist.org/00s0s_32yAwr0KMt...,2017 Porsche Macan GTS Carmine Red w/ Black ...,NaN,ut,41.3112,-111.969
509576,7034389294,https://saltlakecity.craigslist.org/ctd/d/evan...,salt lake city,https://saltlakecity.craigslist.org,12900,2012.0,ram,NaN,NaN,NaN,gas,152296.0,clean,automatic,3C6TD5HT1CG309173,4wd,NaN,pickup,black,https://images.craigslist.org/00E0E_3FtnUA2XPT...,"2012 RAM 2500 4WD Crew Cab 169"" ST"" Offere...",NaN,ut,40.3744,-104.694


In [7]:
cars.describe()

,id,price,year,odometer,county,lat,long
count,5.095770e+05,5.095770e+05,508050.000000,4.172530e+05,0.0,499285.000000,499285.000000
mean,7.044176e+09,5.479684e+04,2009.662236,1.017300e+05,NaN,38.453818,-94.313886
std,4.937219e+06,9.575025e+06,8.567953,1.073790e+05,NaN,5.902152,17.717653
min,6.995212e+09,0.000000e+00,1900.000000,0.000000e+00,NaN,-82.688100,-165.935000
25%,7.040802e+09,3.995000e+03,2007.000000,4.948800e+04,NaN,34.557400,-108.506000
50%,7.045325e+09,9.377000e+03,2011.000000,9.489400e+04,NaN,39.145300,-88.678800
75%,7.048556e+09,1.795500e+04,2015.000000,1.387780e+05,NaN,42.449000,-81.111400
max,7.050103e+09,3.600029e+09,2021.000000,1.000000e+07,NaN,81.569300,94.124800


In [8]:
cars.describe(include=['O'])

,url,region,region_url,manufacturer,model,condition,cylinders,fuel,title_status,transmission,vin,drive,size,type,paint_color,image_url,description,state
count,509577,509577,509577,486813,501588,277643,309894,505592,506515,505858,302152,365434,167574,368046,344871,509563,509561,509577
unique,509577,403,413,43,35852,6,8,5,6,3,180145,3,4,13,12,349468,427803,51
top,https://cedarrapids.craigslist.org/ctd/d/vinto...,fayetteville,https://modesto.craigslist.org,ford,f-150,excellent,6 cylinders,gas,clean,automatic,WDC0G4JB4HF164180,4wd,full-size,sedan,white,https://images.craigslist.org/00i0i_ChcqziILO2...,CLEAN TITLE NO ACCEDINTS 3 YEARS WARRANTY D...,ca
freq,1,4501,2992,92601,11341,140663,111940,441688,487040,457659,114,170482,90481,93225,89322,167,160,52780


## Missing Values and Outliers

I summed up the missing values so we can see the total number for each variable.

In [9]:
# Missing values for each variable
cars.isnull().sum().sort_values()

id                   0
url                  0
region               0
region_url           0
price                0
state                0
image_url           14
description         16
year              1527
title_status      3062
transmission      3719
fuel              3985
model             7989
lat              10292
long             10292
manufacturer     22764
odometer         92324
type            141531
drive           144143
paint_color     164706
cylinders       199683
vin             207425
condition       231934
size            342003
county          509577
dtype: int64

I would also like to show what the percentage of missing values for each variable.

In [10]:
# Percentage of missing values for each variable
round((cars.isnull().sum() / len(cars)) * 100, 4).sort_values()

id                0.0000
url               0.0000
region            0.0000
region_url        0.0000
price             0.0000
state             0.0000
image_url         0.0027
description       0.0031
year              0.2997
title_status      0.6009
transmission      0.7298
fuel              0.7820
model             1.5678
lat               2.0197
long              2.0197
manufacturer      4.4672
odometer         18.1178
type             27.7742
drive            28.2868
paint_color      32.3221
cylinders        39.1860
vin              40.7053
condition        45.5150
size             67.1151
county          100.0000
dtype: float64

In [11]:
# Drop id, url, region_url, image_url, county and vin
cars = cars.drop(['id', 'url', 'region_url', 'image_url', 'county', 'vin'], axis=1)

# Drop rows
cars = cars.dropna(subset=['description'])

I decided to drop ```url```, ```region_url``` and ```image_url``` from the dataset because the URLs don't add any value. In addition, ```id```, ```county``` and ```vin``` were removed. ```county``` only contains missing values and ```id``` and ```vin``` are unique identifiers that don't add any value to the target variable.

I also removed 16 records with missing values from ```description```.

In [12]:
# Print unique values, number of unique values and value frequency details
def variable_details(column):
    print('There are {} missing values in {}\n'.format(cars[column].isnull().sum(), column))
    print('The are {} unique values in {}\n'.format(cars[column].nunique(), column))
    print('The unique values are \n{}\n'.format(cars[column].unique()))
    print('The value frequency is \n{}'.format(cars[column].value_counts()))

I created the ```variable_details``` function to print out number of missing values, unique values, number of unique values and value frequency for each column.

### Missing values for ```year```

In [13]:
# Print out variable details
variable_details('year')

There are 1513 missing values in year

The are 114 unique values in year

The unique values are 
[2012. 2016. 2015. 2018. 2009. 2017. 2007. 2013. 1994. 2004. 2014. 2002.
 2003. 2019. 2010. 2008. 2001.   nan 1992. 2011. 1999. 1935. 2006. 2005.
 1981. 1950. 1955. 1977. 2000. 1951. 1998. 1986. 1995. 1997. 1993. 1991.
 1969. 2020. 1964. 1970. 1980. 1972. 1978. 1990. 1931. 1983. 1996. 1974.
 1971. 1966. 1988. 1952. 1943. 1940. 1956. 1982. 1987. 1959. 1958. 1973.
 1948. 1976. 1984. 1960. 1954. 1962. 1942. 1989. 1968. 1979. 1985. 1965.
 1967. 1963. 1939. 1953. 1929. 1949. 1917. 1947. 1932. 1975. 1946. 1957.
 1961. 1937. 1941. 1938. 1936. 1928. 1934. 1933. 1900. 2021. 1926. 1923.
 1919. 1930. 1920. 1927. 1901. 1902. 1925. 1918. 1912. 1922. 1924. 1945.
 1914. 1916. 1911. 1915. 1921. 1944. 1903.]

The value frequency is 
2016.0    41171
2015.0    37311
2014.0    35626
2013.0    35106
2012.0    33493
2017.0    33450
2011.0    31424
2008.0    28543
2007.0    26193
2010.0    24307
2018.0    22486
2

In [14]:
# Print out variable descriptive statistics
cars['year'].describe()

count    508048.000000
mean       2009.662248
std           8.567967
min        1900.000000
25%        2007.000000
50%        2011.000000
75%        2015.000000
max        2021.000000
Name: year, dtype: float64

In [15]:
# Create missing values boolean series
cars_year_nulls = cars['year'].isnull()

In [16]:
# If first character of description is '2', then the addition of three trailing characters represent year
def populate_year(description):
    desc_slice = str(description)[0]

    if desc_slice == '2':
        year = str(description)[:4]
        return float(year)

In [17]:
# Replace missing values with year from description column
cars.loc[cars_year_nulls, 'year'] = cars.loc[cars_year_nulls, 'description'].apply(populate_year)

In [18]:
# Now see there are 26 missing values
cars['year'].isnull().sum()

26

In [19]:
# Fill remaining 26 missing values with mean
cars['year'] = cars['year'].fillna(cars['year'].mean())

### Missing values for ```title_status```

In [20]:
# Print out variable details
variable_details('title_status')

There are 3048 missing values in title_status

The are 6 unique values in title_status

The unique values are 
['clean' 'rebuilt' 'salvage' 'lien' nan 'missing' 'parts only']

The value frequency is 
clean         487038
rebuilt        10473
salvage         5324
lien            2848
missing          607
parts only       223
Name: title_status, dtype: int64


In [21]:
# Populate missing values using 'missing' categorical value
cars['title_status'] = cars['title_status'].fillna('missing')

### Missing values for ```transmission```

In [22]:
# Print out variable details
variable_details('transmission')

There are 3705 missing values in transmission

The are 3 unique values in transmission

The unique values are 
['manual' 'automatic' 'other' nan]

The value frequency is 
automatic    457657
manual        33962
other         14237
Name: transmission, dtype: int64


In [23]:
# Populate missing values using 'other' categorical value
cars.transmission = cars.transmission.fillna('other')

### Missing values for ```fuel```

In [24]:
# Print out variable details
variable_details('fuel')

There are 3971 missing values in fuel

The are 5 unique values in fuel

The unique values are 
['gas' 'diesel' 'other' nan 'electric' 'hybrid']

The value frequency is 
gas         441686
diesel       41986
other        17100
hybrid        4052
electric       766
Name: fuel, dtype: int64


In [25]:
# Populate missing values using 'other' categorical value
cars['fuel'].fillna('other', inplace=True)

### Missing values for ```lat``` and ```long```

In [26]:
# Print out variable details
variable_details('region')

There are 0 missing values in region

The are 403 unique values in region

The unique values are 
['salt lake city' 'st george' 'lynchburg' 'vermont' 'new river valley'
 'norfolk / hampton roads' 'richmond' 'charlottesville' 'roanoke'
 'danville' 'fredericksburg' 'bellingham' 'harrisonburg'
 'kennewick-pasco-richland' 'moses lake' 'southwest VA' 'winchester'
 'lewiston / clarkston' 'olympic peninsula' 'seattle-tacoma'
 'skagit / island / SJI' 'yakima' 'wenatchee' "spokane / coeur d'alene"
 'madison' 'milwaukee' 'charleston' 'eastern panhandle' 'green bay'
 'kenosha-racine' 'morgantown' 'southern WV' 'west virginia (old)'
 'appleton-oshkosh-FDL' 'eau claire' 'duluth / superior' 'la crosse'
 'janesville' 'wausau' 'wyoming' 'northern WI' 'sheboygan'
 'gadsden-anniston' 'birmingham' 'huntsville / decatur' 'dothan' 'phoenix'
 'little rock' 'anchorage / mat-su' 'mobile' 'montgomery' 'tuscaloosa'
 'auburn' 'flagstaff / sedona' 'mohave county' 'florence / muscle shoals'
 'tucson' 'fairbanks' '

In [27]:
# Print out variable details
variable_details('lat')

There are 10278 missing values in lat

The are 51488 unique values in lat

The unique values are 
[40.7372 40.5881 40.3744 ... 40.758  38.8787 41.3112]

The value frequency is 
43.1824    2339
47.6561    2146
40.4688    2077
46.2348    2057
33.7865    1814
           ... 
39.1549       1
33.5253       1
33.9747       1
40.8451       1
27.3230       1
Name: lat, Length: 51488, dtype: int64


In [28]:
# Print out variable details
variable_details('long')

There are 10278 missing values in long

The are 51467 unique values in long

The unique values are 
[-111.858 -111.884 -104.694 ... -115.226 -111.625 -111.97 ]

The value frequency is 
-84.1122     2339
-117.2370    2146
-74.2817     2077
-119.1280    2057
-84.4454     1813
             ... 
-149.8740       1
-85.3480        1
-87.2190        1
-121.5470       1
-78.9648        1
Name: long, Length: 51467, dtype: int64


In [29]:
# Retrieve region mean for latitude and longitude
region_mean = cars.groupby(['region'])[['lat', 'long']].mean()
region_mean

,lat,long
region,,
SF bay area,37.671302,-121.562285
abilene,32.657045,-95.447892
akron / canton,40.984845,-81.539258
albany,42.294080,-74.202993
albuquerque,35.296387,-106.334763
altoona-johnstown,40.400644,-78.674503
amarillo,34.944558,-101.340845
ames,41.805734,-93.124032
anchorage / mat-su,61.059055,-149.427051


In [30]:
# Create region mean lookup dictionary
region_mean_lookup = region_mean.to_dict('index')
region_mean_lookup

{'SF bay area': {'lat': 37.671301646655316, 'long': -121.56228548884987},
 'abilene': {'lat': 32.65704451403889, 'long': -95.44789200863936},
 'akron / canton': {'lat': 40.98484520641404, 'long': -81.53925789832859},
 'albany': {'lat': 42.29407989317023, 'long': -74.20299275085844},
 'albuquerque': {'lat': 35.296386586206665, 'long': -106.33476334482806},
 'altoona-johnstown': {'lat': 40.40064394463673, 'long': -78.67450311418698},
 'amarillo': {'lat': 34.94455783879523, 'long': -101.34084464127558},
 'ames': {'lat': 41.8057340248963, 'long': -93.12403195020745},
 'anchorage / mat-su': {'lat': 61.05905485502326, 'long': -149.42705097774652},
 'ann arbor': {'lat': 42.518077801724104, 'long': -83.8487925646553},
 'annapolis': {'lat': 38.8226169491525, 'long': -76.43835734463279},
 'appleton-oshkosh-FDL': {'lat': 44.135732051875635,
  'long': -88.61503594256575},
 'asheville': {'lat': 35.33552466369941, 'long': -82.61862472799206},
 'ashtabula': {'lat': 41.70146029411766, 'long': -81.2321

In [31]:
# Create list of row indexes for rows with a missing value for latitude
lat_nulls = cars[cars['lat'].isnull()].index.tolist()

In [32]:
# Create list of row indexes for rows with a missing value for longitude
long_nulls = cars[cars['long'].isnull()].index.tolist()

In [33]:
# Comparing latitude-longitude row indexes to verify lat-long pair is null
def lat_long_combination(lat, long):
    
    print('Latitude length is {}'.format(len(lat)))
    print('Longitude length is {}'.format(len(long)))
    
    is_not_valid = 0
    
    for i in range(len(lat)):
        
        if lat[i] != long[i]:
            is_not_valid += 1
            
    print('There were {} invalid combinations found'.format(str(is_not_valid)))

In [34]:
lat_long_combination(lat_nulls, long_nulls)

Latitude length is 10278
Longitude length is 10278
There were 0 invalid combinations found


In [35]:
# Update latitude missing value with region mean
def retrieve_lat(region):
    latitude = region_mean_lookup[region]['lat']
    return latitude

In [36]:
# Update longitude missing value with region mean
def retrieve_long(region):
    longitude = region_mean_lookup[region]['long']
    return longitude

In [37]:
# Retrieving row indexes lat-long null pairs
cars_lat_long = cars[cars[['lat', 'long']].isnull().any(axis=1)]

In [38]:
# Loop through row indexes and update latitude and longitude values
for i in cars_lat_long.index:
    cars['lat'][i] = retrieve_lat(cars['region'][i])
    cars['long'][i] = retrieve_long(cars['region'][i])

### Missing values for ```manufacturer```

In [39]:
# Print out variable details
variable_details('manufacturer')

There are 22749 missing values in manufacturer

The are 43 unique values in manufacturer

The unique values are 
['volkswagen' 'ford' 'gmc' 'ram' 'toyota' 'chevrolet' 'mercedes-benz'
 'jeep' nan 'honda' 'lexus' 'cadillac' 'dodge' 'hyundai' 'subaru' 'bmw'
 'chrysler' 'nissan' 'mitsubishi' 'buick' 'acura' 'kia' 'rover'
 'alfa-romeo' 'mazda' 'saturn' 'pontiac' 'volvo' 'jaguar' 'fiat' 'audi'
 'mini' 'infiniti' 'lincoln' 'mercury' 'tesla' 'harley-davidson' 'porche'
 'land rover' 'aston-martin' 'ferrari' 'datsun' 'hennessey' 'morgan']

The value frequency is 
ford               92600
chevrolet          73661
toyota             38159
nissan             26435
ram                26177
honda              25585
jeep               23951
gmc                23126
dodge              19344
bmw                13468
hyundai            12182
mercedes-benz      11569
subaru             10259
volkswagen         10214
chrysler            8866
kia                 8821
cadillac            7304
buick          

In [40]:
# Couple minor replacements
cars['manufacturer'] = cars['manufacturer'].replace('mercedes-benz', 'mercedes')
cars['manufacturer'] = cars['manufacturer'].replace('porche', 'porsche')

In [41]:
# Create list of unique values
unique_list = cars['manufacturer'].unique().tolist()

In [42]:
# Look for manufacturer in description and if found update manufacturer
def populate_manufacturer(description):

    # Create word cloud object
    wc = WordCloud().generate(str(description)[:1000])
    
    # Create a list of keys from word cloud object
    keys = [key.lower().strip('_.*') for key in wc.words_.keys()]
    
    # If unique value is found return unique value
    for unique in unique_list:
        if unique in keys:
            return unique

In [43]:
# Create missing values boolean series
cars_manu_nulls = cars['manufacturer'].isnull()

In [44]:
# Create list of row indexes
cars_manu_nulls_list = cars[cars_manu_nulls].index.tolist()

In [45]:
# Drop rows from description
cars = cars.drop(cars[cars['description'] == 'a'].index, axis=0)

In [65]:
# Update missing values in manufacturer with value from description
cars.loc[cars_manu_nulls_list, 'manufacturer'] = cars.loc[cars_manu_nulls_list, 'description'].apply(populate_manufacturer)

KeyError: '[163763] not in index'

### Missing values for ```odometer```

In [46]:
# Print out variable details
variable_details('odometer')

There are 92308 missing values in odometer

The are 119873 unique values in odometer

The unique values are 
[6.3500e+04 1.0000e+01 7.5540e+03 ... 3.9457e+04 8.3200e+05 6.9730e+03]

The value frequency is 
0.0         2268
150000.0    1161
140000.0    1138
130000.0    1115
160000.0    1059
            ... 
45894.0        1
91786.0        1
183571.0       1
121871.0       1
71542.0        1
Name: odometer, Length: 119873, dtype: int64


In [47]:
# Print out variable descriptive statistics
cars['odometer'].describe()

count    4.172510e+05
mean     1.017298e+05
std      1.073792e+05
min      0.000000e+00
25%      4.948800e+04
50%      9.489200e+04
75%      1.387780e+05
max      1.000000e+07
Name: odometer, dtype: float64

In [48]:
# Populate missing values with odometer median value
cars['odometer'] = cars['odometer'].fillna(cars['odometer'].median())

### Missing values for ```type```

In [49]:
# Print out variable details
variable_details('type')

There are 141515 missing values in type

The are 13 unique values in type

The unique values are 
['hatchback' nan 'pickup' 'SUV' 'sedan' 'truck' 'wagon' 'van' 'coupe'
 'convertible' 'other' 'offroad' 'mini-van' 'bus']

The value frequency is 
sedan          93224
SUV            92300
pickup         49547
truck          49509
coupe          19106
other          13631
hatchback      12850
wagon          10937
van            10579
convertible     7737
mini-van        7170
offroad          742
bus              712
Name: type, dtype: int64


In [50]:
# Populate missing values using 'other' categorical value
cars['type'] = cars['type'].fillna('other')

### Missing values for ```drive```

In [51]:
# Print out variable details
variable_details('drive')

There are 144128 missing values in drive

The are 3 unique values in drive

The unique values are 
['4wd' nan 'fwd' 'rwd']

The value frequency is 
4wd    170480
fwd    127189
rwd     67762
Name: drive, dtype: int64


In [52]:
# Populate missing values using 'missing'
cars['drive'] = cars['drive'].fillna('unknown')

### Missing values for ```paint_color```

In [53]:
# Print out variable details
variable_details('paint_color')

There are 164690 missing values in paint_color

The are 12 unique values in paint_color

The unique values are 
['black' nan 'white' 'silver' 'brown' 'blue' 'grey' 'red' 'custom'
 'purple' 'yellow' 'green' 'orange']

The value frequency is 
white     89321
black     70580
silver    51352
blue      34584
red       34410
grey      33289
green      9192
custom     8894
brown      7758
yellow     2533
orange     2125
purple      831
Name: paint_color, dtype: int64


In [54]:
# Populate missing values using 'other'
cars['paint_color'] = cars['paint_color'].fillna('unknown')

### Missing values for ```cylinders```

In [55]:
# Print out variable details
variable_details('cylinders')

There are 199668 missing values in cylinders

The are 8 unique values in cylinders

The unique values are 
['4 cylinders' nan '8 cylinders' '6 cylinders' '10 cylinders'
 '5 cylinders' '3 cylinders' '12 cylinders' 'other']

The value frequency is 
6 cylinders     111939
8 cylinders      95370
4 cylinders      95308
5 cylinders       2714
10 cylinders      2299
other             1506
3 cylinders        562
12 cylinders       193
Name: cylinders, dtype: int64


In [56]:
# Populate missing values using 'other' categorical value
cars['cylinders'] = cars['cylinders'].fillna('other')

### Missing values for ```condition```

In [57]:
# Print out variable details
variable_details('condition')

There are 231918 missing values in condition

The are 6 unique values in condition

The unique values are 
['excellent' nan 'good' 'like new' 'fair' 'new' 'salvage']

The value frequency is 
excellent    140662
good          97260
like new      28166
fair           9461
new            1406
salvage         686
Name: condition, dtype: int64


In [58]:
# Populate missing values using 'unknown'
cars['condition'] = cars['condition'].fillna('unknown')

### Missing values for ```model``` and ```size```

In [59]:
# Print out variable details
variable_details('model')

There are 7975 missing values in model

The are 35850 unique values in model

The unique values are 
['golf r' 'f-150' 'sierra 1500' ... 'Camaro 2-door coupe'
 'Isuzu VehiCROSS' 'peterbilt 378']

The value frequency is 
f-150                              11341
silverado 1500                      7405
1500                                7367
silverado                           5256
2500                                4439
                                   ...  
2005 Optima 27 seat transit bus        1
silverado k1500hd                      1
wrangler unlmtd. sahara 4x4            1
quadcab longbox diesel                 1
civic si mugen                         1
Name: model, Length: 35850, dtype: int64


In [60]:
# Print out variable details
variable_details('size')

There are 341986 missing values in size

The are 4 unique values in size

The unique values are 
['compact' nan 'mid-size' 'full-size' 'sub-compact']

The value frequency is 
full-size      90480
mid-size       47756
compact        25221
sub-compact     4116
Name: size, dtype: int64


In [61]:
cars = cars.drop(['model', 'size'], axis=1)

Due to the number of unique values in ```model``` and the number of missing values in ```size``` along with some of the characteristics from ```model``` and ```size``` being represented by ```type```, I decided to drop ```model``` and ```size```.

### Outliers 

In [62]:
def outlier_visual(column):
    plt.figure(figsize=(10, 5))
    plt.boxplot(cars[column])
    plt.title('{} Boxplot'.format(column))
    plt.show

## Data Exploration and Feature Engineering 

### Univariate Analysis 

#### Continous Variables 

In [ ]:
def continuous_visual(column):
    plt.figure(figsize=(15, 5))
    plt.hist(cars[column])
    plt.title('{} Histogram'.format(column))
    plt.show()

#### Categorical Variables 

In [69]:
def categorical_visual(column):
    plt.figure(figsize=(15, 5))
    plt.barh(cars.groupby(column)[column].count().index,
             cars.groupby(column)[column].count())
    plt.title('Horizontal bar chart for {}'.format(column))
    plt.show()

### Multivariate Analysis 

## Model Creation and Performance Evaluation 